In [ ]:
#Instalar MLFLOW y PYNGROK
#!pip install mlflow

In [1]:
#Verificar si MLFLOW está instalado
!mlflow

Usage: mlflow [OPTIONS] COMMAND [ARGS]...

Options:
  --version  Show the version and exit.
  --help     Show this message and exit.

Commands:
  artifacts    Upload, list, and download artifacts from an MLflow...
  db           Commands for managing an MLflow tracking database.
  deployments  Deploy MLflow models to custom targets.
  doctor       Prints out useful information for debugging issues with MLflow.
  experiments  Manage experiments.
  gc           Permanently delete runs in the `deleted` lifecycle stage.
  models       Deploy MLflow models locally.
  recipes      Run MLflow Recipes and inspect recipe results.
  run          Run an MLflow project from the given URI.
  runs         Manage runs.
  sagemaker    Serve models on SageMaker.
  server       Run the MLflow tracking server.


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')

%matplotlib inline


In [ ]:
import mlflow
import mlflow.sklearn

mlflow.set_experiment('ToyotaCorolla')

Importar Dataset

In [ ]:

file_name = 'ToyotaCorolla.csv'
df_raw = pd.read_csv(file_name, engine='python')

print(f'Dimensiones del dataset: {df_raw.shape}')


Visualizar columnas del dataset

In [ ]:
df_raw.columns

Ver informacion y tipo de dato de cada columna

In [ ]:
df_raw.info()

Otra manera de ver los tipos de datos

In [ ]:
data_types = df_raw.dtypes
print(data_types)

Convertir los nombres de las columnas a minúscula

In [ ]:
df_raw.columns = df_raw.columns.str.lower()

Verificación de que los nombres de las columnas fueron modificadas a minuscula

In [ ]:
df_raw.columns

In [ ]:
df_raw.describe(include='all').T

Eliminar las primeras columnas innecesarias

In [ ]:
columns_to_delete = ['id', 'model', 'cylinders'] #, 'mfg_month', 'mfg_year']
df = df_raw.drop(columns=columns_to_delete, axis=1)
df = pd.DataFrame(df)

In [ ]:
df.info()

In [ ]:
df.describe(include='all').T

Buscamos valores nulos

In [ ]:
df.isnull().sum()

Buscamos valores duplicados

In [ ]:
df[df.duplicated()]

In [ ]:
#Dada la duplicación de un registro, es temprano para decidir si eliminarlo o no.

Mostrar la variable objetivo

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 7))
fig.suptitle("Distribution of Price")
ax1.hist(df.price)
ax1.set_xlabel(df.price.name)

ax2.boxplot(df.price)
ax2.set_xlabel(df.price.name)

plt.show()

In [ ]:
# Se puede observar que el histograma presenta una forma similar a una distribución normal, pero con un alargamiento hacía la derecha.
# Esta distribución presenta un sesgo positivo (o a la derecha), en donde la media tendrá un valor mayor que la mediana, y la mediana tendrá un valor mayor que la moda.

Mostrar la variable objetivo con mayor zoom

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 7))
fig.suptitle("Distribution of Price")
ax1.hist(df.price, bins=60)
ax1.set_xlabel(df.price.name)

ax2.boxplot(df.price)
ax2.set_xlabel(df.price.name)

plt.show()

#En esta ocasión se configura el parámetro bins en 60, el cual por defecto es 10, y representa el número de contenedores que serán representados en el histograma.

In [ ]:
# Ahora buscaremos el valor máximo a partir del cual la variable objetivo (precio) deja de caer dentro del extremo superior
# del diagrama de caja-bigotes (boxplots)

sns.boxplot(x="km", y="price", data=df)
max_value = plt.gca().lines[4].get_ydata().max()
print("Valor máximo:", max_value)

In [ ]:
registros_mayores = (df['price'] > max_value).sum()

# Imprimir el número de registros mayores al valor umbral
print("Número de registros mayores a", max_value, ":", registros_mayores)

Variables categoricas

In [ ]:
variables_categoricas = ['mfg_year', 'fuel_type', 'hp', 'met_color', 'automatic', 'cc', 'doors', 'gears',
                         'quarterly_tax', 'mfr_guarantee', 'bovag_guarantee', 'guarantee_period', 'abs', 'airbag_1',
                         'airbag_2' , 'airco', 'automatic_airco', 'boardcomputer', 'cd_player', 'central_lock',
                         'powered_windows', 'power_steering', 'radio', 'sport_model', 'backseat_divider',
                         'metallic_rim', 'radio_cassette']

fig = plt.figure(figsize=(32,25))

for i in range(len(variables_categoricas)):
  continuos = variables_categoricas[i]

  ax1 = fig.add_subplot(7,4,i+1)
  ax1.set_xlabel(continuos)

  df.groupby(continuos).size().plot(kind='bar')

In [ ]:
# Algunas variables binarias quedan sin ser representadas ya que son demasiadas. (tow_bar y mistlamps)

Variables continuas

In [ ]:
variables_continuas = []

fig = plt.figure(figsize=(34,25))

for i in range(len(variables_continuas)):
  continuos = variables_continuas[i]

  ax1 = fig.add_subplot(3,5,i+1)
  ax1.set_xlabel(continuos)

  df[continuos].plot(kind='box')

In [ ]:
variables_continuas_scatter = ['age_08_04','km','hp','quarterly_tax','weight','cc']

fig = plt.figure(figsize=(34,25))

for i in range(len(variables_continuas_scatter)):
    aux = variables_continuas_scatter[i]

    ax1 = fig.add_subplot(6,5,i+1)
    ax1.set_xlabel(aux)

    plt.scatter(df[aux],df.price)

In [ ]:
#Podemos observar como las variables hp, quartely_tax no son continuas, es decir son discretas.
#Sin embargo podemos observar que la variable cc a primera vista parece continua, pero esto se debe a las proporciones del
# del eje x (debido al outliers), en la gráfica anterior se puede observar como cc es un valor discreto y no continuo

In [ ]:
#eliminar columnas por falta de significancia estadistica
#columns_to_delete = ['doors', 'gears', 'cc', 'mfr_guarantee', 'met_color', 'automatic', 'guarantee_period']
#df.drop(columns=columns_to_delete, axis=1, inplace=True)

In [ ]:
dummies = pd.get_dummies(data=df, columns=['fuel_type'], drop_first=True) # convierto las variables dummies
pd.concat([df, dummies], axis=1)
df.drop(columns=["fuel_type"], axis=1, inplace=True)

In [ ]:
split_param = {
    'test_size': 0.4,
    'random_state': 42,
    'shuffle': True
}

x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(df.drop(['price'], axis=1), df.price, **split_param)
x_train_1.shape, x_test_1.shape, y_train_1.shape, y_test_1.shape

Primera ejecución de una regresión con todas las variables.

In [ ]:
#Primera ejecución de una regresión con todas las variables.
model = sm.OLS(y_train_1, sm.add_constant(x_train_1)).fit()
print(model.summary())

In [ ]:
#Linea Base

#Calculamos RMSE (error cuadratico medio)
y_pred_1 = model.predict(sm.add_constant(x_test_1))
RMSE = mean_squared_error(y_test_1,y_pred_1, squared = 'false')
print(f'RMSE: {RMSE}')

 Tracking del primer modelo arrojado por la regresión (baseline)

* R2: 0.912
* R2-ajustado: 0.909
* kurtosis: 5.359
* skew: 0.069
* durbin-watson: 1.990
* RMSE: 1376324.3921468968

In [ ]:
dw = print(round(sm.stats.stattools.durbin_watson(model.resid),3))

In [ ]:
with mlflow.start_run():
  #mlflow.log_metric("accuracy", accuracy)
  mlflow.log_params({"R2": model.rsquared, "R2-Ajustado": model.rsquared_adj, "Skew": model.resid.skew, "Kurtosis": model.resid.kurtosis, "Durbin-Watson": dw, "MSE - RMSE": RMSE})

  mlflow.sklearn.log_model(model, "modelo 1")

# Análisis de la regresión y decisión sobre las variables

In [ ]:
# met_color -> t = 0.605 y P>|t| = 0.546
# Falta de significancia estadística: El valor "t" se utiliza para evaluar si el coeficiente es estadísticamente
# diferente de cero. En este caso, un valor "t" de 0.605 sugiere que el coeficiente no es muy diferente de cero.
# Si el valor "t" es cercano a cero, esto indica que la variable independiente correspondiente puede no estar
# contribuyendo significativamente a la predicción del resultado. Esto podría sugerir que la variable no es relevante
# en el modelo y podría eliminarse.

# Alto valor p: El valor "P>|t|" se utiliza para evaluar si el coeficiente es estadísticamente significativo.
# Un valor p alto, como 0.546, sugiere que no hay suficiente evidencia para rechazar la hipótesis nula de que el
# coeficiente es igual a cero. En otras palabras, la variable independiente no tiene un efecto estadísticamente
# significativo en el resultado. Un valor p alto es una indicación de que la variable podría no ser importante en el
# modelo.

#Siguiendo esta linea las variables a eliminar son:

# met_color ->  t = 0.605 y  P>|t| = 0.546
# cc ->  t = -1.193 y  P>|t| = 0.233
# gears ->  t = 0.763  y  P>|t| = 0.446
# airbag_1 ->  t = 0.556  y  P>|t| = 0.578
# airbag_2 ->  t = 0.441  y  P>|t| = 0.659
# cd_player ->  t = 1.440  y  P>|t| = 0.150
# central_lock -> t = -1.062  y  P>|t| = 0.289
# power_steering -> t = -0.467 y  P>|t| = 0.641
# radio -> t = -1.120  y  P>|t| = 0.263
# mistlamps -> t = 0.247 y  P>|t| = 0.805
# backseat_divider -> t = -0.955  y  P>|t| = 0.340
# metallic_rim -> t = 0.219  y  P>|t| =  0.827
# radio_cassette -> t = 1.027  y  P>|t| = 0.305

# Comienzo nuevo modelo - MODELO 2

In [ ]:
columns_to_delete = ['met_color', 'cc', 'gears', 'airbag_1', 'airbag_2', 'cd_player', 'central_lock',
                     'power_steering', 'radio', 'mistlamps', 'backseat_divider', 'metallic_rim', 'radio_cassette' ] #, 'mfg_month', 'mfg_year']
df2 = df.drop(columns=columns_to_delete, axis=1)
df2 = pd.DataFrame(df2)

In [ ]:
split_param = {
    'test_size': 0.4,
    'random_state': 42,
    'shuffle': True
}

x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(df2.drop(['price'], axis=1), df2.price, **split_param)
x_train_2.shape, x_test_2.shape, y_train_2.shape, y_test_2.shape

In [ ]:
#Segunda ejecución de una regresión con todas las variables.
model2 = sm.OLS(y_train_2, sm.add_constant(x_train_2)).fit()
print(model2.summary())

In [ ]:
#Calculamos RMSE (error cuadratico medio)
y_pred_2 = model2.predict(sm.add_constant(x_test_2))
RMSE = mean_squared_error(y_test_2,y_pred_2,squared = 'false')
print(f'RMSE: {RMSE}')

 Tracking del segundo modelo

* R2: 0.911
* R2-ajustado: 0.909
* kurtosis: 5.460
* skew: 0.040
* durbin-watson: 1.978
* RMSE: 1367838.6712162532

In [ ]:
dw = print(round(sm.stats.stattools.durbin_watson(model2.resid),3))

In [ ]:
with mlflow.start_run():
  #mlflow.log_metric("accuracy", accuracy)
  mlflow.log_params({"R2": model2.rsquared, "R2-Ajustado": model2.rsquared_adj, "Skew": model2.resid.skew, "Kurtosis": model2.resid.kurtosis, "Durbin-Watson": dw, "MSE - RMSE": RMSE})

  mlflow.sklearn.log_model(model, "modelo 2")

In [ ]:
#Siguiendo esta linea las variables a eliminar son:

# mfg_year
# mfg_month
# automatic
# doors
# mfr_guarantee

# Comienzo nuevo modelo - MODELO 3

In [ ]:
columns_to_delete = ['mfg_year', 'mfg_month', 'automatic', 'doors', 'mfr_guarantee']
df3 = df2.drop(columns=columns_to_delete, axis=1)
df3 = pd.DataFrame(df3)

In [ ]:
split_param = {
    'test_size': 0.4,
    'random_state': 42,
    'shuffle': True
}

x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(df3.drop(['price'], axis=1), df3.price, **split_param)
x_train_3.shape, x_test_3.shape, y_train_3.shape, y_test_3.shape

In [ ]:
#Tercera ejecución de una regresión con todas las variables.
model3 = sm.OLS(y_train_3, sm.add_constant(x_train_3)).fit()
print(model3.summary())

In [ ]:
#Calculamos RMSE (error cuadratico medio)
y_pred_3 = model3.predict(sm.add_constant(x_test_3))
RMSE = mean_squared_error(y_test_3,y_pred_3, squared = 'false')
print(f'RMSE: {RMSE}')

Tracking del segundo modelo

* R2: 0.899
* R2-ajustado: 0.897
* kurtosis: 4.636
* skew: -0.039
* durbin-watson: 2.007
* RMSE: 1438163.9047859225

In [ ]:
dw = print(round(sm.stats.stattools.durbin_watson(model3.resid),3))

In [ ]:
with mlflow.start_run():
  #mlflow.log_metric("accuracy", accuracy)
  mlflow.log_params({"R2": model3.rsquared, "R2-Ajustado": model3.rsquared_adj, "Skew": model3.resid.skew, "Kurtosis": model3.resid.kurtosis, "Durbin-Watson": dw, "MSE - RMSE": RMSE})

  mlflow.sklearn.log_model(model, "modelo 3")

In [ ]:
#Siguiendo esta linea las variables a eliminar son:

# boardcomputer

# Comienzo nuevo modelo - MODELO 4

In [ ]:
columns_to_delete = ['boardcomputer']
df4 = df3.drop(columns=columns_to_delete, axis=1)
df4 = pd.DataFrame(df4)

In [ ]:
split_param = {
    'test_size': 0.4,
    'random_state': 42,
    'shuffle': True
}

x_train_4, x_test_4, y_train_4, y_test_4 = train_test_split(df4.drop(['price'], axis=1), df4.price, **split_param)
x_train_4.shape, x_test_4.shape, y_train_4.shape, y_test_4.shape

In [ ]:
#Cuarta ejecución de una regresión con todas las variables.
model4 = sm.OLS(y_train_4, sm.add_constant(x_train_4)).fit()
print(model4.summary())

In [ ]:
#Calculamos RMSE (error cuadratico medio)
y_pred_4 = model4.predict(sm.add_constant(x_test_4))
RMSE = mean_squared_error(y_test_4,y_pred_4, squared = 'false')
print(f'RMSE: {RMSE}')

In [ ]:
dw = print(round(sm.stats.stattools.durbin_watson(model4.resid),3))

In [ ]:
with mlflow.start_run():
  #mlflow.log_metric("accuracy", accuracy)
  mlflow.log_params({"R2": model4.rsquared, "R2-Ajustado": model4.rsquared_adj, "Skew": model4.resid.skew, "Kurtosis": model4.resid.kurtosis, "Durbin-Watson": dw, "MSE - RMSE": RMSE})

  mlflow.sklearn.log_model(model, "modelo 4")


TEST

In [ ]:
testDf = df4

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 7))
fig.suptitle("Distribution of Price")
ax1.hist(testDf.price, bins=60)
ax1.set_xlabel(testDf.price.name)

ax2.boxplot(testDf.price)
ax2.set_xlabel(testDf.price.name)

plt.show()

In [ ]:
sns.boxplot(x="km", y="price", data=testDf)
max_value = plt.gca().lines[4].get_ydata().max()
print("Valor máximo:", max_value)

In [ ]:
# Crear una máscara booleana para los registros que cumplen la condición
condicion = testDf['price'] <= 17847.5

# Filtrar el DataFrame para mantener solo los registros que cumplen la condición
df_filtrado = testDf[condicion]

# Mostrar el DataFrame resultante
df_filtrado.info()

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 7))
fig.suptitle("Distribution of Price")
ax1.hist(df_filtrado.price, bins=80)
ax1.set_xlabel(df_filtrado.price.name)

ax2.boxplot(df_filtrado.price)
ax2.set_xlabel(df_filtrado.price.name)

plt.show()

In [ ]:
variables_continuas_scatter = ['age_08_04','km','hp','quarterly_tax','weight']

fig = plt.figure(figsize=(34,25))

for i in range(len(variables_continuas_scatter)):
    aux = variables_continuas_scatter[i]

    ax1 = fig.add_subplot(6,5,i+1)
    ax1.set_xlabel(aux)

    plt.scatter(df_filtrado[aux],df_filtrado.price)

# 5to Modelo

In [ ]:
split_param = {
    'test_size': 0.4,
    'random_state': 42,
    'shuffle': True
}

x_train_5, x_test_5, y_train_5, y_test_5 = train_test_split(df_filtrado.drop(['price'], axis=1), df_filtrado.price, **split_param)
x_train_5.shape, x_test_5.shape, y_train_5.shape, y_test_5.shape

In [ ]:
#Cuarta ejecución de una regresión con todas las variables.
model5 = sm.OLS(y_train_5, sm.add_constant(x_train_5)).fit()
print(model5.summary())

In [ ]:
#Calculamos RMSE (error cuadratico medio)
y_pred_5 = model5.predict(sm.add_constant(x_test_5))
RMSE = mean_squared_error(y_test_5,y_pred_5, squared = 'false')
print(f'RMSE: {RMSE}')

In [ ]:
dw = print(round(sm.stats.stattools.durbin_watson(model5.resid),3))

In [ ]:
with mlflow.start_run():
  #mlflow.log_metric("accuracy", accuracy)
  mlflow.log_params({"R2": model5.rsquared, "R2-Ajustado": model5.rsquared_adj, "Skew": model5.resid.skew, "Kurtosis": model5.resid.kurtosis, "Durbin-Watson": dw, "MSE - RMSE": RMSE})

  mlflow.sklearn.log_model(model, "modelo 5")

In [ ]:
plt.figure(figsize=(12,12))
plt.title("Mapa de Correlaciones")
corr = df_filtrado.corr(method="pearson")
sns.heatmap(corr, annot=True, cmap=plt.cm.viridis, linewidths=0.1, linecolor="white", square=True)